<a href="https://colab.research.google.com/github/Mohamed28112003/Muffakir/blob/main/Muffakir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [172]:
!pip install  transformers
!pip install langchain
!pip install -U langchain-community
!pip install unstructured
!pip install "unstructured[pdf]"
!apt-get install -y poppler-utils
!pip install LangChain -q
!pip install -U langchain-community -q
!pip install langchain transformers
!pip install chromadb
!pip install sentence-transformers

!pip install langchain-nvidia-ai-endpoints
!pip install PyPDF2
!pip install rank-bm25 sentence-transformers



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [173]:
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from dotenv import dotenv_values
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


# Load TXT

In [189]:
# Load the text file instead of PDF
with open("/content/الجوانب_العملية_فى_الاجراءات_الجنائية_2016_extracted.txt", "r", encoding="utf-8") as file:
    pages_arabic = file.readlines()

# Concatenate the text into a single string, removing extra white spaces
my_text = ""
for line in pages_arabic:
    my_text += line.strip() + " "  # strip() removes leading/trailing white spaces, add a space to preserve word separation

# Now `my_text` contains the entire content of the text file without extra white spaces
print(my_text)


--- Page 1 --- الجوانب العملية في الإجراءات الجنائية )الحلول النموذجية للقضايا - صيغ الأوراق القضائية(  اللدكتور جميل عبد الباقى الصغير دكتورا الدولة ف لون الجا جامعة رين رسا  المحامى لدى المحاكم الجنائية 5    --- Page 2 --- الجزء الأول الحلول النموذجية للقضايا الأكثر شيوعا في الواقع العملي   --- Page 3 --- خصائص النيابة العامة القضية ‎(١)‏  أصدر وزير العدل أمره الى عضو النيابة العامة بعدم إقامة الدعوى الجنانية  بالنسبة الى متهم معين« ومع ذلك قام العضو برفعهاء فما رأيك في هذا الإجراء؟ الإجابة  المسائل القانونية التي تثيرها القضية : تتعلق هذه القضية بخاصية التبعية التدريجية كإحدى خصائص النيابة العامة.  الأسانيد القانونية : من المقرر قانونا أن رجال النيابة العامة تابعون لرؤسائهم دون غيرهم بترتيب درجاتهم ثم لوزير العدل )م57 من قانون السلطة القضائية(. فللوزير حق الإشراف والرقابة على أعضاء النيابة العامة للتأكد من قيامهم .بواجباتهم الوظيفية طبقا للقانون.  وتظهر رناسة الوزير الإدارية في تنقلات أعضاء النيابة وتحديد محال إقامتهم واختصاصهم المكاني والنوعي وندبهم من مكان لآخرء علاوة على ما له من

### TF - IDF

In [ ]:
# import os
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from langchain.document_loaders import TextLoader


# # Apply TF-IDF using TfidfVectorizer
# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(my_text)

# # Get the feature names (terms) and convert the matrix to a DataFrame
# feature_names = vectorizer.get_feature_names_out()
# tfidf_df = pd.DataFrame(data=X.toarray(), columns=feature_names)

# # Print the TF-IDF DataFrame
# print(tfidf_df.head())


# Read Csv

In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/BoodyAhmedHamdy/EgyLaw-Squad/" + splits["train"])

In [ ]:
import json
# Function to clean and extract the text from the answer column
def clean_answer(answer):
    # If the answer is already a dictionary
    if isinstance(answer, dict):
        return answer['text'][0]
    # If the answer is a JSON-like string, parse it
    elif isinstance(answer, str):
        answer_dict = json.loads(answer)
        return answer_dict['text'][0]
    return None  # In case it's neither, return None

# Apply the cleaning function to the 'answer' column
df['answers'] = df['answers'].apply(clean_answer)


In [ ]:
import pandas as pd

# يفترض أن df هو DataFrame الخاص بك
# دمج الأعمدة الثلاثة (السؤال، الإجابة، والسياق) في عمود واحد جديد
df['merged'] = df.apply(lambda row: f"السؤال: {row['question']}\nالإجابة: {row['answers']}\nالسياق: {row['context']}", axis=1)




In [ ]:
df =df['merged']

In [ ]:
df.to_csv('law.csv', index=False)

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/law.csv")
data = loader.load()

In [ ]:
print(data[0].metadata)

In [ ]:
data = data + documents

In [ ]:
print(data[711].metadata)

In [ ]:
print(data[711].page_content)

In [ ]:
len(data)

In [ ]:
print(data[91].page_content)

# Spilting

In [190]:




text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Adjust chunk size based on content length
    chunk_overlap=200,  # No overlap needed
    separators=["--- Page"] # Split at answer or next question
)

In [191]:
documents = [ my_text]
metadatas = [ {"document":'الاختصاص القضائي د.مينا عادل_extracted.txt'} ]

In [192]:
recursive_chuncks =text_splitter_recursive.create_documents(
    documents, metadatas=metadatas
)

In [193]:
len(recursive_chuncks)

351

In [194]:
print(recursive_chuncks[1].page_content)

--- Page 3 --- خصائص النيابة العامة القضية ‎(١)‏  أصدر وزير العدل أمره الى عضو النيابة العامة بعدم إقامة الدعوى الجنانية  بالنسبة الى متهم معين« ومع ذلك قام العضو برفعهاء فما رأيك في هذا الإجراء؟ الإجابة  المسائل القانونية التي تثيرها القضية : تتعلق هذه القضية بخاصية التبعية التدريجية كإحدى خصائص النيابة العامة.  الأسانيد القانونية : من المقرر قانونا أن رجال النيابة العامة تابعون لرؤسائهم دون غيرهم بترتيب درجاتهم ثم لوزير العدل )م57 من قانون السلطة القضائية(. فللوزير حق الإشراف والرقابة على أعضاء النيابة العامة للتأكد من قيامهم .بواجباتهم الوظيفية طبقا للقانون.  وتظهر رناسة الوزير الإدارية في تنقلات أعضاء النيابة وتحديد محال إقامتهم واختصاصهم المكاني والنوعي وندبهم من مكان لآخرء علاوة على ما له من توجيه التنبيه لأعضاء النيابة الذين يخلون بواجباتهم بعد سماع أقوال عضو النيابة )م571 من قانون السلطة القضانية(.  كما تظهر سلطة وزير العدل الرناسية على النائب العام الوزير بالقضايا الهامة.  .ولكن ليس لوزير العدل أي إشراف قضاني على أعضاء النيابة العامة. وبناء على ذلك لا يجوز للوزير أن يأمر أحد أ

# Embeddding

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

print("loading embedding model and index ....")

model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model_id = "asafaya/bert-large-arabic"

embeddings = HuggingFaceEmbeddings(
    model_name = embedding_model_id,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [195]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings



# Apply the embeddings to the documents (recursive_chunks)
print("Embedding documents ....")

# Initialize Chroma vector store
vector_store = Chroma(
    collection_name="Book",  # Give your collection a meaningful name
    embedding_function=embeddings
)

# Add documents and their embeddings to ChromaDB
vector_store.add_texts(
    texts=[doc.page_content for doc in recursive_chuncks],  # Get the content from chunks
    metadatas=metadatas  # Optionally pass metadata
)

print("Documents and embeddings have been stored in ChromaDB.")


Embedding documents ....
Documents and embeddings have been stored in ChromaDB.


In [ ]:
#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

In [196]:
qna_template = "\n".join([
    "Act as a knowledgeable legal analyst. Analyze the provided legal context and respond to the subsequent question with thoroughness and clarity. If the information needed to answer the question is not present in the context, respond with 'NO ANSWER IS AVAILABLE.'",
    "give the answer in details with egyptian accent ",

    "### Context",
    "{context}",

    "### Question",
    "{question}",

    "### Answer:",
])

qna_prompt = PromptTemplate(
    template=qna_template,
    input_variables=['context', 'question'],
    verbose=True
)

stuff_chain = load_qa_chain(llm, chain_type="stuff", prompt=qna_prompt)


In [201]:
# Example: Query for the most similar document
query_text = '''
أمر النائب العام أحد أعضاء
النيابة لم يستجب لذلك وأصدر قرارا بأن لا وجه لإقامة الدعوى؛ ايه رأيك في الحوار ده

'''
query_embedding = embeddings.embed_query(query_text)

# Perform a similarity search
results = vector_store.similarity_search_by_vector(query_embedding, k=6)  # Retrieve top 5 most similar documents

for result in results:
    print(f"Document: {result.page_content}\nMetadata: {result.metadata}")


Document: --- Page 4 --- أمر النائب العام أحد أعضاء |! النيابة« لم يستجب لذلك وأصدر قرارا بأن لا وجه لإقامة الدعوى؛ فما رأيك في هذا الإجراء؟  الإجابة المسائل القانونية التي تثيرها القضية : تتعلق أيضا هذه القضية بخاصية التبعية  : يتبع أعضاء النيابة - عدا نيابة النقض - النائب العام تبعية إدارية. وقضائية. فهو يشرف إداريا على جميع أعضاء النيابة. وهو الأصيل في تحريك ومباشرة الدعوى الجنائية. بمعنى أن باقي أعضاء النيابة يعتبرون وكلاء عنه يستعملون الدعوى باسمه ويستمدون منه سلطاتهم.  ومن أجل ذلك يكون للنائب العام أن يصدر ما يراه من تعليمات الى أعضاء النيابة بشأن إقامة أو مباشرة الدعوى العمومية دون التحقيق فيهاء وأن يأمر بحفظ التحقيق أو بأن لا وجه لاقامة الدعوى العمومية. ويترتب على مخالفة أعضاء النيابة العامة لهذه التعليمات والأوامر بطلان تصرفاتهم؛ لآن في ذلك خروج عن حدود الوكالة الممنوحة لهم من النانب العام؛ فيكون تصرفه صادرا من غير ذي صفة؛ وذلك علاوة على مسنوليته الإدارية.  التطبيق : بإنزال المبادئ القانونية سالفة الذكر فإنه إذا أمر النائب العام أحد أعضاء النيابة برفع الدعوى الجنانية: فقام هذا

In [202]:
answer = stuff_chain(
    {
        "input_documents": results,
        "question": query_text
    },
    return_only_outputs=True,
)



In [203]:
print(answer)

{'output_text': 'النائب العام هو الأصيل في تحريك ومباشرة الدعوى الجنائية، وله سلطة إصدار التعليمات لأعضاء النيابة بشأن إقامة أو مباشرة الدعوى العمومية دون التحقيق فيها. وأمر النائب العام لأحد أعضاء النيابة برفع الدعوى الجنائية، فإذا قام هذا الأخير بحفظها أو أصدر أمراً بعدم وجود وجه لإقامة الدعوى، فإن هذا الأمر الصادر من العضو يكون باطلاً.\n\nالسبب في ذلك هو أن أعضاء النيابة يعتبرون وكلاء عن النائب العام، ويستمدون سلطاتهم منه. فإذا خالفوا تعليماته أو أوامره، فإن تصرفاتهم تكون باطلة لخروجها عن حدود الوكالة الممنوحة لهم من النائب العام. علاوة على ذلك، يكون تصرف العضو في هذه الحالة صادراً من غير ذي صفة، مما يترتب عليه بطلان الإجراء.\n\nبالتالي، يعتبر إجراء عضو النيابة في هذه الحالة غير صحيح، ويجب أن يتبع تعليمات النائب العام فيما يتعلق بإقامة الدعوى الجنائية.'}


In [205]:
# استخرج النص من الإجابة
output_text = answer['output_text']

# تنسيق النص لإظهار الفقرات بشكل واضح
formatted_output = output_text.replace(":", ":\n").replace("\n", "\n\n")

# طباعة النص المنسق
print(formatted_output)


النائب العام هو الأصيل في تحريك ومباشرة الدعوى الجنائية، وله سلطة إصدار التعليمات لأعضاء النيابة بشأن إقامة أو مباشرة الدعوى العمومية دون التحقيق فيها. وأمر النائب العام لأحد أعضاء النيابة برفع الدعوى الجنائية، فإذا قام هذا الأخير بحفظها أو أصدر أمراً بعدم وجود وجه لإقامة الدعوى، فإن هذا الأمر الصادر من العضو يكون باطلاً.



السبب في ذلك هو أن أعضاء النيابة يعتبرون وكلاء عن النائب العام، ويستمدون سلطاتهم منه. فإذا خالفوا تعليماته أو أوامره، فإن تصرفاتهم تكون باطلة لخروجها عن حدود الوكالة الممنوحة لهم من النائب العام. علاوة على ذلك، يكون تصرف العضو في هذه الحالة صادراً من غير ذي صفة، مما يترتب عليه بطلان الإجراء.



بالتالي، يعتبر إجراء عضو النيابة في هذه الحالة غير صحيح، ويجب أن يتبع تعليمات النائب العام فيما يتعلق بإقامة الدعوى الجنائية.


# Hypothetical Document Embeddings

In [206]:
# reusing the question and vectorstore as earlier
from langchain.prompts import ChatPromptTemplate

# HyDE document genration prompt
template = "\n".join([
    "Act as a knowledgeable legal analyst. Analyze the provided legal context and respond to the subsequent question with thoroughness and clarity.",
    "give the answer in details in arabic only ",


    "### Question",
    "{question}",

    "### Answer:",
])
prompt_hyde = ChatPromptTemplate.from_template(template)

# generating a hypothetical document based on the user input
hypothetical_document = llm.invoke(prompt_hyde.format(question=query_text))

In [207]:
print(hypothetical_document.content)

بالنظر إلى الحوار المذكور، يمكن تحليله من منظور قانوني على النحو التالي:

أولاً: صلاحيات النائب العام:

النائب العام هو رأس الهرم في النيابة العامة، وله صلاحيات واسعة في إدارة النيابة العامة وإصدار القرارات المتعلقة بالتحقيقات والاتهامات. ووفقاً للقانون، فإن النائب العام له الحق في إصدار أوامر وتوجيهات إلى أعضاء النيابة العامة.

ثانياً: قرار النائب العام:

في الحوار المذكور، أصدر النائب العام أمراً لأحد أعضاء النيابة العامة لم يستجب لذلك. هذا القرار يعتبر قراراً إداريًا، ويلزم العضو بالامتثال له.

ثالثاً: قرار عضو النيابة العامة:

رد عضو النيابة العامة على قرار النائب العام بإصدار قراراً بأن لا وجه لإقامة الدعوى. هذا القرار يعتبر قراراً قضائيًا، ويتعلق بتقدير أدلة الاتهام ووجود الأسباب الكافية لإقامة الدعوى.

رابعاً: التقييم القانوني:

من الناحية القانونية، يمكن القول إن عضو النيابة العامة لم يمتثل لقرار النائب العام، والذي يعتبر قراراً إداريًا ملزماً. ومع ذلك، فإن عضو النيابة العامة لديه سلطة قضائية في تقدير الأدلة واتخاذ القرارات المتعلقة بإقامة الدعوى.

خامساً: النتيجة:

في ضوء ما س

In [208]:
# retrieving relevant document based on hyde document instead of user query
retriever = vector_store.as_retriever()
retireved_docs = retriever.invoke(hypothetical_document.content)

In [209]:
len(retireved_docs)

4

In [210]:
print(retireved_docs[1].page_content)

--- Page 38 --- 8-5  القضية )51(  ة مكافحة المخدرات أن المدعو )أ( المسجل تاجر مخدرات بالإدارة ينوي بيع كمية من الأفيون أثناء جلوسه في المقهى. توجه الضابط الى المقهى وجلس متنكرا بجواره وبعد لحظات شاهد )أ( يسلم حقيبة مغلقة الى )ب( ويتقاضى منه مبلغا من النقودء فقبض على الاثنينء وقام بتفتيش الحقيبة فوجد بها. كمية من الأفيون. توجه الضابط بعد ذلك الى منزل كل من الاثنين وفتشهما فوجد في منزل )أ( كمية من الحشيشء وعثر في منزل )ب( على سلاح بدون ترخيص.  اعترف )أ( للضابط بأنه حصل على هذه المخدرات من )ج( فتوجه الى منزل هذا الأخير للقبض عليه. وفتش المنزل فعثر على كميات من الحشيش والأفيون. وتولت النيابة التحقيق« وحضر محام وطلب الاطلاع على الأوراق قبل استجواب المتهم فرفض وكيل النيابة. اعترف المتهمون )أ( )ب(؛ )ج( بحيازة المخدرات ثم أنكروا التهمة أمام المحكمة؛ ما رأيك في الإجراءات التي تمت؟ دعم إجابتك بالأسانيد القانونية؟  المسائل القانونية التي تثيرها القضية : تثير القضية الحالية البحث في عدة .موضوعات وهي : سلطة مأمور الضبط القضائي في جمع الاستدلالات« حالة التلبس والسلطات التي تمنحها لمأمور الضبط اعتراف

In [211]:
qna_template = "\n".join([
    "Act as a knowledgeable legal analyst. Analyze the provided legal context and respond to the subsequent question with thoroughness and clarity. If the information needed to answer the question is not present in the context, respond with 'NO ANSWER IS AVAILABLE.'",
    "give the answer in details with egyptian accent ",

    "### Context",
    "{context}",

    "### Question",
    "{question}",

    "### Answer:",
])

In [212]:
qna_prompt = PromptTemplate(
    template=qna_template,
    input_variables=['context', 'question'],
    verbose=True
)

stuff_chain = load_qa_chain(llm, chain_type="stuff", prompt=qna_prompt)

In [213]:
answer = stuff_chain(
    {
        "input_documents": retireved_docs,
        "question": query_text
    },
    return_only_outputs=True,
)

answer

{'output_text': 'لا يجوز للوزير أن يأمر أحد أعضاء النيابة العامة بالتصرف في الدعوى الجنائية أو عدم التصرف فيها. ولا يترتب على مخالفة أوامر وزير العدل في صدد الدعوى العمومية أي بطلان. \n\nبإنزال المبادئ القانونية والأحكام القضائية سالفة الذكر يتضح أن قيام عضو النيابة برفع الدعوى الجنائية إلى المحكمة يكون صحيحا ويتعين على المحكمة قبوله لأن رئاسة وزير العدل على أعضاء النيابة رئاسة إدارية فحسب لا يترتب عليها أي أثر قانوني.'}

In [214]:

# استخرج النص من الإجابة
output_text = answer['output_text']

# تنسيق النص لإظهار الفقرات بشكل واضح
formatted_output = output_text.replace(":", ":\n").replace("\n", "\n\n")

# طباعة النص المنسق
print(formatted_output)


لا يجوز للوزير أن يأمر أحد أعضاء النيابة العامة بالتصرف في الدعوى الجنائية أو عدم التصرف فيها. ولا يترتب على مخالفة أوامر وزير العدل في صدد الدعوى العمومية أي بطلان. 



بإنزال المبادئ القانونية والأحكام القضائية سالفة الذكر يتضح أن قيام عضو النيابة برفع الدعوى الجنائية إلى المحكمة يكون صحيحا ويتعين على المحكمة قبوله لأن رئاسة وزير العدل على أعضاء النيابة رئاسة إدارية فحسب لا يترتب عليها أي أثر قانوني.


In [ ]:
# prompt = ChatPromptTemplate.from_template(template)

# answer = llm.invoke(prompt.format(context=retireved_docs,question=qna_template))
# print(answer.content)

# Query Translation Rag Fusion

In [215]:
from langchain.retrievers.multi_query import MultiQueryRetriever

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(), llm=llm
)

In [216]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [217]:
# setting the logger to listen to specific tasks
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
# Generating multiple contexts for the same question and retrieving the relevant contexts
unique_docs = retriever_from_llm.invoke(query_text)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three alternative versions of the user question:', 'ما هو رأيك في رفض النائب العام لمتابعة قضية إحدى أعضاء النيابة؟', 'كيف تتعامل مع قرار النائب العام بعدم إقامة دعوى ضد عضو في النيابة؟', 'ما هي الآثار المحتملة لقرار النائب العام بعدم متابعة قضية عضو النيابة؟']


8

In [218]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

final_docs = reciprocal_rank_fusion(unique_docs)
len(final_docs)

11

In [219]:
final_docs

[(['id', None], 0.13333333333333333),
 (['metadata', {}], 0.13114754098360656),
 (['type', 'Document'], 0.12698412698412698),
 (['page_content',
   "--- Page 85 --- ثائيَا : اتجاه الفقه الحديث المؤيد للإحالة الدولية  أيد الفقه الحديث في مصر فكرة الإحالة الدولية « خاصة بعد تأييد الفقه والقضاء الغربيين لها. واستتد الفقه على العديد من المبررات وهي  كالتالي :  \u200e-١\u200f لا تتعارض الإحالة الدولية مع فكرة السياد فقبول جميع الدول بفكرة الإحالة الدولية ستدعم التعاون الدولي وبالتالي المعاملة بالمثل'.  \u200f1- جواز الإحالة الدولية أ  \u200fالإحالة الداخلية هناك العديد من الأمور لم يرد بشأنها نص فيما يتعلق بالاختصاص  \u200fالقضائي الدولي وتنفيذ الأحكام الأ  \u200fت الاستعانة بالنصوص ال  \u200fاللمسألة داخليا لتشمل الشق الدوني'.  \u200fهشام صادق, القانون الدولى الخاص؛ دار المطبوعات الجديدف 05:50  5\u200e    "],
  0.016129032258064516),
 (['page_content',
   "--- Page 32 --- أولا : يتفق هذا الاستثناء مع مبدأ سيادة الدولة على إقليمها واحتزام. لسيادة الدول الأخرى ولتفادي عدم تنفيذ الأحكام الصاد